<a href="https://colab.research.google.com/github/ruthenian8/forhse2/blob/master/%D0%9A%D0%BB%D1%8E%D1%87%D0%B5%D0%B2%D1%8B%D0%B5_%D1%81%D0%BB%D0%BE%D0%B2%D0%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## импорты

In [ ]:
!python --version

Python 3.10.12


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
!pip install wheel
!pip install --upgrade "pymorphy2[fast] @ git+https://github.com/pymorphy2-fork/pymorphy2.git"
import pymorphy2
# from pymystem3 import Mystem
# mystem_analyzer = Mystem(grammar_info=False, disambiguation=False)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from collections import Counter
from nltk.stem import SnowballStemmer
from sklearn.svm import SVC
from sklearn.metrics.pairwise import cosine_similarity
import os

  Cloning https://github.com/pymorphy2-fork/pymorphy2.git to /tmp/pip-install-hhwt3s6r/pymorphy2_fcd53d4e91ef4aa58e0f60d9a2fb8714
  Running command git clone --filter=blob:none --quiet https://github.com/pymorphy2-fork/pymorphy2.git /tmp/pip-install-hhwt3s6r/pymorphy2_fcd53d4e91ef4aa58e0f60d9a2fb8714
  Resolved https://github.com/pymorphy2-fork/pymorphy2.git to commit eb93d32f41687d5a907ce7a0ee0b515657e67787
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --upgrade gensim

## ABSTRACT TABLE CLASS

In [ ]:
class table:
    def __init__(self, filepath):
        self._filepath = filepath;
        if ".xlsx" in filepath:
            self.df = pd.read_excel(self._filepath)
        elif ".csv" in filepath:
            self.df = pd.read_csv(self._filepath)
        else:
            raise ValueError("Класс принимает на вход только файлы .csv и .xlsx")
        self.length = self.df.shape[0]

                        # "([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)"
        self.tokenizer = RegexpTokenizer(r'\w+') # EXPRESSION FOR TOKENIZING
        self.tokens = [self.tokenizer.tokenize(n.lower()) for n in self.df["текст"]]
        self.morph=pymorphy2.MorphAnalyzer() # FOR FILTERING CLITICS
        self.stemmer = SnowballStemmer(language='russian')

        self.important = {'NOUN', 'VERB', 'PRTF', \
                          'ADJF', 'INFN', 'COMP', \
                          'PRTS', 'NUMR', 'ADVB', \
                          'GRND', 'PRED'} # A LIST OF IMPORTANT POS TAGS FOR PYMORPHY

        self.cash = {} # CASH FOR LEMMATIZING
        self.lemmatized = ""
        self.stemized = ""

        self.tfrepr = ""; # WILL BE DEFINED BY THE DICTIONARY CLASS
        self.countrepr = "";
        self.vectrepr = "";

    def __dictAdd__(self, wrd):
        if wrd in self.cash:
            return self.cash[wrd]

        form=self.morph.parse(wrd)[0]
        if self.fc==True:
            wtag = str(form.tag)
            if wtag[:wtag.find(",")] not in self.important:
                return ""

        if self.type == "lemm":
            normal = form.normal_form
            self.cash[wrd] = normal
            return normal
        else:
            normal = self.stemmer.stem(wrd)
            self.cash[wrd] = normal
            return normal

    def __converter__(self, common):
        for idx, text in enumerate(self.tokens):
            sent = []
            for word in text:
                sent.append(self.__dictAdd__(word))

            sent.append(self.df["программа"][idx]) # add PROGRAM to the word list
            sent.append(self.df["программа"][idx])
            common.append(" ".join([n for n in sent]))

    def lemmatize(self, filterclitics=False):
        if "Леммы" in self.df.columns:
            self.lemmatized = self.df["Леммы"]
        else:
            new = []
            self.type = "lemm"

            if filterclitics: # WITHOUT IRRELEVANT PARTS OF SPEECH
                self.fc = True
                self.__converter__(new)

            else: # INCLUDING ALL PARTS OF SPEECH
                self.fc = False
                self.__converter__(new)

            self.lemmatized = new
            self.df["Леммы"] = self.lemmatized
            new_filepath = "new_" + self._filepath
            self.df.to_excel(new_filepath) if self._filepath.endswith(".xlsx") else self.df.to_csv(new_filepath)
            self.cash = {}


    def stemize(self, filterclitics=False):
        new = []
        self.type = "stem"

        if filterclitics: # WITHOUT IRRELEVANT PARTS OF SPEECH
            self.fc = True
            self.__converter__(new)

        else: # INCLUDING ALL PARTS OF SPEECH
            self.fc = False
            self.__converter__(new)

        self.stemized = new
        self.cash = {}

## SPECIFIC TABLE CLASSES

In [ ]:
class trainTable(table):
    def __init__(self, filepath):
        super().__init__(filepath)
        self.words = []

    def standartizeY(self):
        try:
            self.df["ключевые слова"]
        except:
            raise ValueError("The table doesn't have a keyword field")

        self.wordSets = []
        for sentence in self.df["ключевые слова"].tolist():
            sentence = str(sentence).replace("\t", "").replace(",", ", ").replace("  ", " ").replace("  ", " ")
            sentence = sentence.split(", ") #разделяем набор ключевых слов
            for word in sentence: #добавляем в список уникальные значения ключевых слов
                self.words.append(word)
            self.wordSets.append(set(sentence))
        c = Counter(self.words)
        self.words = [n[0] for n in c.most_common(450)]

class testTable(table):
    def __init__(self, filepath):
        super().__init__(filepath)
        self.results = [[] for n in range(self.length)]

    def clearResults(self):
        self.results = [[] for n in range(self.length)]

    def update(self, wrd, answers, thresh=0.35):
        for n in np.where(answers > thresh)[0]:
            self.results[int(n)].append(wrd)

    def save(self):
        if not self.results:
            raise Exception("No prediction has been applied yet")
        else:
            self.df["ключевые слова"] = self.results
        self.df.to_excel(("new_" + self._filepath))

## DICTIONARY CLASS

In [ ]:
class dictionary:
    def __init__(self, table1, table2, typ="lemm"):
        if not (str(type(table1)) == "<class '__main__.trainTable'>" \
        and str(type(table2)) == "<class '__main__.testTable'>"):
            raise ValueError("Both inputs should belong to the table class \n Pass the train table first")
        self.tab1 = table1
        self.tab2 = table2

        self.CV = CountVectorizer()
        self.tf = TfidfVectorizer()

        if typ == "lemm":
            try:
                self.dict = self.tab1.lemmatized + self.tab2.lemmatized
            except:
                raise Exception("Both tables should be lemmatized first")
        elif typ == "stem":
            try:
                self.dict = self.tab1.stemized + self.tab2.stemized
            except:
                raise Exception("Both tables should be stemmed first")
        else:
            raise ValueError("The 'typ' argument should be specified as 'lemm' or 'stem'")
#         self.dict = pd.concat((self.tab1.df["текст"], self.tab2.df["текст"]))

        self.count = ""
        self.tfized = ""

    def countize(self):
        self.count = self.CV.fit_transform(self.dict)
        self.tab1.countrepr = self.count[:self.tab1.length,:]
        self.tab2.countrepr = self.count[self.tab1.length:,:]

    def tfize(self):
        self.tfized = self.tf.fit_transform(self.dict)
        self.tab1.tfrepr = self.tfized[:self.tab1.length,:]
        self.tab2.tfrepr = self.tfized[self.tab1.length:,:]

In [ ]:
class predictor:
    def __init__(self, wrd, table1, table2, method="main"):
        self.word = wrd
        self.tab1 = table1
        self.tab2 = table2
        self.method = method
        self.prediction = 0

        # y для таблицы c кл. словами
        self.y = [n for n in map(lambda x: int(self.word in x), self.tab1.wordSets)]

        if self.method == "main":
            self.train_data = self.tab1.tfrepr
            self.objective = self.tab2.tfrepr
        elif self.method == "keywords":
            self.train_data = self.tab1.LDArepr
            self.objective = self.tab2.LDArepr
        elif self.method == "vectors":
            self.train_data = self.tab1.vectrepr
            self.objective = self.tab2.vectrepr

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.train_data, \
                                self.y, \
                                test_size=0.05, \
                                random_state=42)

    def logreg(self, wei=0.8):
        self.clf = LogisticRegression(penalty="l2", \
                                      class_weight="balanced", \
                                      solver="liblinear")
        self.clf.fit(self.X_train, self.y_train)
        prediction = self.clf.predict(self.X_test)
        f1 = f1_score(y_true=self.y_test, y_pred=prediction, average="macro")
        print(f"f1 = {str(f1)}")
        result = self.clf.predict_proba(self.objective)[:,1]# picking solely the second part of the tuple

        self.updateres(res=result, weight=wei)

    def XGBoost(self, wei=0.4):
        XG_pred = xgb.DMatrix(self.objective)
        self.XG_train = xgb.DMatrix(self.X_train, label=self.y_train)
        self.XG_test = xgb.DMatrix(self.X_test, label=self.y_test)
        param = {'max_depth': 2, \
                 'eta': 1, \
                 'objective': 'binary:logistic', \
                 'nthread': 6, \
                 'eval_metric':'logloss'}
        evallist = [(self.XG_test, 'eval'), (self.XG_train, 'train')]
        num_round = 12
        self.bst = xgb.train(param, self.XG_train, \
                             num_round, \
                             evallist, \
                             early_stopping_rounds=10, \
                             verbose_eval=False)
        result = self.bst.predict(XG_pred)
        self.updateres(res=result, weight=wei)

    def computeCosine(self, x):
#         return cosine_similarity(np.atleast_2d(x), \
#                                  np.atleast_2d(self.centroid))[0][0]
        return cosine_similarity(x, \
                                 self.centroid)[0][0]

    def batchCosine(self, train, test, objective):
        return (np.array([a for a in map(lambda x: self.computeCosine(x), train)]).reshape((-1, 1)), \
                np.array([b for b in map(lambda x: self.computeCosine(x), test)]).reshape((-1, 1)), \
                np.array([c for c in map(lambda x: self.computeCosine(x), objective)]).reshape((-1, 1)))

    def axisMethod(self, wei=0.4):
        pos_pol = self.X_train[np.where(np.array(self.y_train, dtype="int32") == 0)]
        pos_pol = np.mean(pos_pol, axis=0)

        neg_pol = self.X_train[np.where(np.array(self.y_train, dtype="int32") == 1)]
        neg_pol = np.mean(neg_pol, axis=0)
        self.centroid = pos_pol - neg_pol

        batch = self.batchCosine(self.X_train, self.X_test, self.objective)
        new_X_train, new_X_test, new_objective = batch[0], batch[1], batch[2]


        self.predictor = LogisticRegression(penalty="l2", \
                                      class_weight="balanced", \
                                      solver="liblinear")
        self.predictor.fit(new_X_train, self.y_train)
        prediction = self.predictor.predict(new_X_test)
        f1 = f1_score(y_true=self.y_test, y_pred=prediction, average="macro")
        print(f"f1 = {str(f1)}")

        result = self.predictor.predict_proba(new_objective)[:,1]# picking solely the second part of the tuple
        self.updateres(res=result, weight=wei)

    def SVM(self, wei=0.3):
        self.svc = SVC(C=1.0, \
                       kernel='poly', \
                       degree=2, \
                       gamma="auto", \
                       class_weight='balanced')
        self.svc.fit(self.X_train, self.y_train)
        prediction = self.svc.predict(self.X_test)
        self.f1 = f1_score(y_true=self.y_test, y_pred=prediction, average="macro")
        print(f"f1 = {str(self.f1)}")
        result = self.svc.predict(self.objective)
        self.updateres(res=result, weight=wei)

    def updateres(self, res, weight=0.5):
        if type(self.prediction) == int:
            self.prediction = res
        else:
            final = np.array([self.prediction * (1-weight), res * weight])
            final = np.sum(final, axis=0) / 2
            self.prediction = final

    def resout(self):
        print(f"done with: {self.word}\n")
        if type(self.prediction) != int:
            return self.word, self.prediction
        else:
            raise Exception("No prediction has been applied yet")

## Дополнительные опции: метод осей, LDA - for COLAB only

In [ ]:
# !pip install --upgrade simple_elmo
# from simple_elmo import ElmoModel
# !wget "http://vectors.nlpl.eu/repository/20/196.zip"
# !unzip "196.zip"
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import fasttext
!wget "https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz"


In [ ]:
class dictWithOptions(dictionary):
    def __init__(self, table1, table2, typ):
        super().__init__(table1, table2, typ)

#     def LDAize(self):
#         self.tf = TfidfVectorizer()
#         self.tf.fit(self.dict)

# #       CODE BELOW STOLEN FROM HABR.COM
# #       https://habr.com/ru/post/417167/
#         self.idfs = self.tf.idf_
#         lower_thresh = 2.
#         upper_thresh = 7.
#         not_often = self.idfs > lower_thresh
#         not_rare = self.idfs < upper_thresh
#         mask = not_often * not_rare
#         #self.good_words = np.array(self.tf.get_feature_names())[mask]

#         self.tab1.LDArepr = ""
#         self.tab2.LDArepr = ""

    def vectorize(self):
        if type(self.tab1.lemmatized) == str or \
        type(self.tab1.lemmatized) == str:
            raise Exception("Both tables should be lemmatized")

        # if not ElmoModel:
        #     raise Exception("Failed to find the 'ElmoModel' module")
        # model = ElmoModel()
        # #use only in colab
        # link = os.getcwd()
        # model.load(link)
        model = fasttext.load_facebook_vectors("araneum_none_fasttextcbow_300_5_2018.tgz")
        self.tab1.tokens = [n.split(" ") for n in self.tab1.lemmatized]
        self.tab2.tokens = [n.split(" ") for n in self.tab2.lemmatized]
        self.tab1.vectrepr = [[model[a] for a in n] for n in self.tab1.tokens]
        self.tab2.vectrepr = [[model[a] for a in n] for n in self.tab2.tokens]

        self.tab1.vectrepr = np.mean(model.get_elmo_vectors(self.tab1.tokens), axis=1)
        self.tab2.vectrepr = np.mean(model.get_elmo_vectors(self.tab2.tokens), axis=1)

In [ ]:
def test(file1, file2, options=""):
    first = trainTable(file1)
    second = testTable(file2)
    print("import done")

    first.lemmatize(filterclitics=True)
    second.lemmatize(filterclitics=True)
    first.standartizeY()
    print("preprocessing done")

    common = dictionary(first, second, "lemm")
    common.tfize()
    print("vectorizing done")

    for n in first.words:
        prd = predictor(n, first, second, "main")
        prd.logreg()
        prd.XGBoost()
        prd.SVM()
        prd.axisMethod()
        two = prd.resout()
        second.update(two[0], two[1])
    second.df["ключевые слова"] = second.results
    print(second.df.head(25))

In [ ]:
!head -n 5 merged.csv

﻿программа,вопрос,село,год,собиратель1,собиратель2,собиратель3,собиратель4,текст,информант1,информант2,информант3,информант4
XXIa,12 е,Плосское-Карповская,2023,ВК,МСК,ААВ,,"[Когда у вас на кладбище ходят?] Мы, ну обы\чц+но хо\дят два\дцать пе\рвоё… июля\. У нас Проко\пьев день счц+ита\ется. Вели\кий пра\здник. Всю жи\зь. Всю жи\зь. [Соб.: Расскажите про Прокопия, мы ее не знаем.] Дак это… Ра\ньше была\ леге\нда, ли как ли, я не зна\ю, это са\мое, что плы\у% младе\нец по реке\ и приста\у% там в… в У\стье [местная река] там на э\ту, на пра\вую сто\рону. Мол, его\ подня\ли, вот в э\ту це\рков подня\ли [в Бестужеве был церковный комплекс, теперь чц+уть ниже стоит небольшая деревянная церковь], и вот э\то как ра\з его\ подня\ли что\ ли два\дцать пе\рвоё ию\ля. Та\к вот говори\ли-то. Его\ ста\ли э\то сам… Э\то уж от веко\в ве\чц+ных, не зна\ю я, де\ды и праде\д и всё вре\мя этот пра\здник отмечц+а\ли. В сове\тское вре\мя быва\ло, когда\ то\ что са\мо-то горя\чц+ее вре\мя бы\ло пятидеся\тые г

In [ ]:
second = testTable("merged.xlsx")
first = trainTable("66.xlsx")
print("import done")

first.lemmatize(filterclitics=True)
second.lemmatize(filterclitics=True)
first.standartizeY()
print("preprocessing done")

common = dictionary(first, second, "lemm")
common.tfize()
print("vectorizing done")

import done
preprocessing done
vectorizing done


In [ ]:
for n in first.words:
    prd = predictor(n, first, second, "main")
    prd.logreg(0.8)
    # prd.XGBoost(0.3)
#     prd.SVM()
    # prd.axisMethod(0.2)
    two = prd.resout()
    second.update(two[0], two[1])
print(second.results)
second.df["ключевые слова"] = second.results

f1 = 0.7738682092555332
done with: Пространство

f1 = 0.8308205608575698
done with: Дом

f1 = 0.9038166134940329
done with: Праздники

f1 = 0.7461014795316329
done with: Время

f1 = 0.9258097980924067
done with: Календарь

f1 = 0.8816713333048347
done with: Семейные обряды

f1 = 0.8212299255777515
done with: Народное православие

f1 = 0.7887029349225406
done with: Запреты

f1 = 0.8639406965884744
done with: Ребенок

f1 = 0.928087032611988
done with: Скот

f1 = 0.7280979008814968
done with: Пища

f1 = 0.8631854256854257
done with: Демонология

f1 = 0.7320143151180255
done with: Тело человека

f1 = 0.7941279024786259
done with: Животное

f1 = 0.8276955082958501
done with: Вода

f1 = 0.7404959427365088
done with: Женщина

f1 = 0.7745457183272277
done with: Растения

f1 = 0.6979933471463164
done with: Хороший-плохой

f1 = 0.8930923418493386
done with: Похоронный обряд

f1 = 0.8652431346840237
done with: Покойник

f1 = 0.7175692783503014
done with: Родство

f1 = 0.7533109579100145
done with

In [ ]:
second.df['ключевые слова'] = [list(set(i)) for i in second.df['ключевые слова']]

In [ ]:
second.df.to_excel("sunday.xlsx")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cp /content/drive/MyDrive/66.xlsx .
# !cp /content/drive/MyDrive/sunday/merged.xlsx .
!cp ./sunday.xlsx /content/drive/MyDrive/sunday/

## Старые наработки (временно)

In [ ]:
resulti = []
for n in range(X_pred_txt.shape[0]):
    resulti.append([])

In [ ]:
def writedown_2(a, answers, thresh=0.45):
    for n in np.where(answers > thresh)[0]:
        resulti[int(n)].append(a)

In [ ]:
for n in words:
    #results = Klslovo(n)
    results = Klslovo_merged(n)
    #writedown(results[0], results[1])
    writedown_2(results[0], results[1])

In [ ]:
## Про конкатенацию номеров программы

In [ ]:
#progr1 = df["программа"]
#progr2 = df2["программа"]
#prvocab = pd.concat((progr1, progr2))

In [ ]:
#progr1 = df["программа"]
#progr2 = df2["программа"]
#prvocab = pd.concat((progr1, progr2))
#OHE = OneHotEncoder()
#prvocab = OHE.fit_transform(prvocab)
#CV = CountVectorizer()
#prvocab = CV.fit_transform(prvocab)
#tf = TfidfTransformer()
#prvocab = tf.fit_transform(prvocab)
#progr1 = prvocab[:31079,:]
#progr2 = prvocab[31079:,:]
#del prvocab

In [ ]:
# X_tr_txt = df["текст"]
# X_pred_txt = df2["текст"]

In [ ]:
# vocab = pd.concat((X_tr_txt, X_pred_txt))
# CV = CountVectorizer()
# vocab = CV.fit_transform(vocab)
# tf = TfidfTransformer()
# vocab = tf.fit_transform(vocab)

In [ ]:
        if len(self.X_train.shape) == 3: # Для трёхмерных элмо-векторов:

            new_X_train = np.mean(self.X_train, axis = 1) # сплющивает предложения до одного вектора
            new_X_test = np.mean(self.X_test, axis = 1)
            new_objective = np.mean(self.objective, axis = 1)

            pos_pol = new_X_train[np.where(self.y_train == 1),:]
            pos_pol = np.mean(pos_pol, axis=0)

            neg_pol = new_X_train[np.where(self.y_train == 0),:]
            neg_pol = np.mean(neg_pol, axis=0)
            self.centroid = pos_pol - neg_pol

            batch = self.batchCosine(new_X_train, new_X_test, new_objective)
            new_X_train, new_X_test, new_objective = batch[0], batch[1], batch[2]

        elif len(self.X_train.shape) == 2: # для двухмерных векторов: tf-idf, countvectorizer